# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [23]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1451590734 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/zephaniahgrunschlag/.graphlab/anaconda/lib/python2.7/site-packages/certifi/cacert.pem
1451590734 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to zg33@columbia.edu and will expire on October 15, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-10195 - Server binary: /Users/zephaniahgrunschlag/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1451590734.log
[INFO] GraphLab Server Version: 1.7.1


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.

#CUTE:
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 1.291472     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 1.315825     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [14]:
print("number non-zero is: %d" % model_all.coefficients['value'].nnz())
model_all.coefficients.print_rows(num_rows=18)

number non-zero is: 6
+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None |  274873.05595 |
|     bedrooms     |  None |      0.0      |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 8468.53108691 |
|   sqft_living    |  None | 24.4207209824 |
| sqft_living_sqrt |  None | 350.060553386 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 842.068034898 |
|    sqft_above    |  None | 20.0247224171 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+-----

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

In [21]:
Q5_A1 = {"SFrame" : model_all.coefficients[model_all.coefficients['value'] != 0]}
Q5_A1["names"] = Q5_A1['SFrame']['name']
Q5_A1
#type(model_all.coefficients)

{'SFrame': Columns:
 	name	str
 	index	str
 	value	float
 
 Rows: Unknown
 
 Data:
 +------------------+-------+---------------+
 |       name       | index |     value     |
 +------------------+-------+---------------+
 |   (intercept)    |  None |  274873.05595 |
 |    bathrooms     |  None | 8468.53108691 |
 |   sqft_living    |  None | 24.4207209824 |
 | sqft_living_sqrt |  None | 350.060553386 |
 |      grade       |  None | 842.068034898 |
 |    sqft_above    |  None | 20.0247224171 |
 +------------------+-------+---------------+
 [? rows x 3 columns]
 Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
 You can use len(sf) to force materialization., 'names': dtype: str
 Rows: ?
 ['(intercept)', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'grade', 'sqft_above', ... ]}

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [22]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [27]:
Q5_A2 = []
for l1_penalty in np.logspace(1, 7, num=13):
    print("l1_penalty = %f" % l1_penalty)
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    y = model.predict(validation)
    error = validation['price'] - y
    RSS = (error*error).sum()
    Q5_A2.append({'l1_penalty': l1_penalty, 'RSS': RSS})

l1_penalty = 10.000000
l1_penalty = 31.622777
l1_penalty = 100.000000
l1_penalty = 316.227766
l1_penalty = 1000.000000
l1_penalty = 3162.277660
l1_penalty = 10000.000000
l1_penalty = 31622.776602
l1_penalty = 100000.000000
l1_penalty = 316227.766017
l1_penalty = 1000000.000000
l1_penalty = 3162277.660168
l1_penalty = 10000000.000000


In [28]:
Q5_A2

[{'RSS': 625766285142460.5, 'l1_penalty': 10.0},
 {'RSS': 625766285362394.4, 'l1_penalty': 31.622776601683793},
 {'RSS': 625766286057885.1, 'l1_penalty': 100.0},
 {'RSS': 625766288257224.4, 'l1_penalty': 316.22776601683796},
 {'RSS': 625766295212186.1, 'l1_penalty': 1000.0},
 {'RSS': 625766317206080.8, 'l1_penalty': 3162.2776601683795},
 {'RSS': 625766386760658.0, 'l1_penalty': 10000.0},
 {'RSS': 625766606749278.5, 'l1_penalty': 31622.776601683792},
 {'RSS': 625767302791634.9, 'l1_penalty': 100000.0},
 {'RSS': 625769507643885.8, 'l1_penalty': 316227.76601683791},
 {'RSS': 625776517727024.5, 'l1_penalty': 1000000.0},
 {'RSS': 625799062845466.6, 'l1_penalty': 3162277.6601683795},
 {'RSS': 625883719085425.0, 'l1_penalty': 10000000.0}]

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [29]:
Q5_A2_1 = 10.0

In [32]:
model_10 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=10.0)
y_test = model_10.predict(testing)
error_test = testing['price'] - y_test
RSS_test = (error_test*error_test).sum()
Q5_A2_2 = RSS_test
Q5_A2_2

156983602381664.12

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [33]:
Q5_A2_3_nnz = model_10.coefficients['value'].nnz()
Q5_A2_3_nnz

18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [34]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [35]:
l1_penalty_values = np.logspace(8, 10, num=20)
l1_penalty_values

array([  1.00000000e+08,   1.27427499e+08,   1.62377674e+08,
         2.06913808e+08,   2.63665090e+08,   3.35981829e+08,
         4.28133240e+08,   5.45559478e+08,   6.95192796e+08,
         8.85866790e+08,   1.12883789e+09,   1.43844989e+09,
         1.83298071e+09,   2.33572147e+09,   2.97635144e+09,
         3.79269019e+09,   4.83293024e+09,   6.15848211e+09,
         7.84759970e+09,   1.00000000e+10])

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [36]:
penalty_and_nnz = []
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=lmodel.coefficients['value'].nnz()1_penalty)
    penalty_and_nnz.append((l1_penalty, ))
penalty_and_nnz

[(100000000.0, 18),
 (127427498.57031322, 18),
 (162377673.91887242, 18),
 (206913808.11147901, 18),
 (263665089.87303555, 17),
 (335981828.62837881, 17),
 (428133239.8719396, 17),
 (545559478.11685145, 17),
 (695192796.17755914, 17),
 (885866790.41008317, 16),
 (1128837891.6846883, 15),
 (1438449888.2876658, 15),
 (1832980710.8324375, 13),
 (2335721469.0901213, 12),
 (2976351441.6313133, 10),
 (3792690190.7322536, 6),
 (4832930238.5717525, 5),
 (6158482110.6602545, 3),
 (7847599703.5146227, 1),
 (10000000000.0, 1)]

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [37]:
l1_penalty_min = 2976351441.6313133
l1_penalty_max = 3792690190.7322536
Q5_A3 = {'l1_penalty_min': l1_penalty_min, 'l1_penalty_max': l1_penalty_max}
Q5_A3

{'l1_penalty_max': 3792690190.7322536, 'l1_penalty_min': 2976351441.6313133}

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [38]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [42]:
def search_for_best_model(l1_penalty_values):
    results = []
    for l1_penalty in l1_penalty_values:
        print("l1_penalty = %f" % l1_penalty)
        model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                                  validation_set=None, verbose=False,
                                                  l2_penalty=0., l1_penalty=l1_penalty)
        y = model.predict(validation)
        error = validation['price'] - y
        RSS = (error*error).sum()
        results.append({'l1_penalty': l1_penalty, 'RSS': RSS, 'nnz': model.coefficients['value'].nnz()})
    return results

In [44]:
results = search_for_best_model(l1_penalty_values)

l1_penalty = 2976351441.631313
l1_penalty = 3019316638.952415
l1_penalty = 3062281836.273518
l1_penalty = 3105247033.594620
l1_penalty = 3148212230.915722
l1_penalty = 3191177428.236824
l1_penalty = 3234142625.557926
l1_penalty = 3277107822.879028
l1_penalty = 3320073020.200130
l1_penalty = 3363038217.521233
l1_penalty = 3406003414.842335
l1_penalty = 3448968612.163437
l1_penalty = 3491933809.484539
l1_penalty = 3534899006.805641
l1_penalty = 3577864204.126743
l1_penalty = 3620829401.447845
l1_penalty = 3663794598.768947
l1_penalty = 3706759796.090049
l1_penalty = 3749724993.411151
l1_penalty = 3792690190.732254


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [45]:
results

[{'RSS': 966925692362085.2, 'l1_penalty': 2976351441.6313133, 'nnz': 10},
 {'RSS': 974019450084554.6, 'l1_penalty': 3019316638.9524155, 'nnz': 10},
 {'RSS': 981188367942453.8, 'l1_penalty': 3062281836.2735176, 'nnz': 10},
 {'RSS': 989328342459473.5, 'l1_penalty': 3105247033.5946198, 'nnz': 10},
 {'RSS': 998783211265890.1, 'l1_penalty': 3148212230.9157219, 'nnz': 10},
 {'RSS': 1008477167020094.4, 'l1_penalty': 3191177428.236824, 'nnz': 10},
 {'RSS': 1018298780553819.4, 'l1_penalty': 3234142625.5579262, 'nnz': 10},
 {'RSS': 1028247992205977.0, 'l1_penalty': 3277107822.8790283, 'nnz': 10},
 {'RSS': 1034616909232827.4, 'l1_penalty': 3320073020.2001305, 'nnz': 8},
 {'RSS': 1038554735941040.2, 'l1_penalty': 3363038217.5212326, 'nnz': 8},
 {'RSS': 1043237237871701.2, 'l1_penalty': 3406003414.8423347, 'nnz': 8},
 {'RSS': 1046937488751711.5, 'l1_penalty': 3448968612.1634369, 'nnz': 7},
 {'RSS': 1051147625612862.5, 'l1_penalty': 3491933809.484539, 'nnz': 7},
 {'RSS': 1055992735342999.4, 'l1_pena

In [46]:
Q5_A4_lowest_RSS_sparsity7_l1penalty = 3448968612.1634369
model_best = graphlab.linear_regression.create(training, target='price', features=all_features,
                                                  validation_set=None, verbose=False,
                                                  l2_penalty=0., l1_penalty=3448968612.1634369)
Q5_A4_lowest_RSS_sparsity7_features = model_best.coefficients[model_best.coefficients['value'] != 0]
Q5_A4_lowest_RSS_sparsity7_features

name,index,value
(intercept),None,222253.192544
bedrooms,None,661.722717782
bathrooms,None,15873.9572593
sqft_living,None,32.4102214513
sqft_living_sqrt,None,690.114773313
grade,None,2899.42026975
sqft_above,None,30.0115753022
